# Machine learning project

#### Downloading the data

In [5]:
%%cmd
mkdir data
wget http://dataincubator-wqu.s3.amazonaws.com/mldata/providers-train.csv -P ./data
wget http://dataincubator-wqu.s3.amazonaws.com/mldata/providers-metadata.csv -P ./data

Microsoft Windows [Version 10.0.18363.418]
(c) 2019 Microsoft Corporation. All rights reserved.

(base) E:\HOCTAP\DATA SCIENCE\WQU COURSE\ML Project>mkdir data

(base) E:\HOCTAP\DATA SCIENCE\WQU COURSE\ML Project>wget http://dataincubator-wqu.s3.amazonaws.com/mldata/providers-train.csv -P ./data

(base) E:\HOCTAP\DATA SCIENCE\WQU COURSE\ML Project>wget http://dataincubator-wqu.s3.amazonaws.com/mldata/providers-metadata.csv -P ./data

(base) E:\HOCTAP\DATA SCIENCE\WQU COURSE\ML Project>

--2023-10-10 07:38:52--  http://dataincubator-wqu.s3.amazonaws.com/mldata/providers-train.csv
Resolving dataincubator-wqu.s3.amazonaws.com (dataincubator-wqu.s3.amazonaws.com)... 3.5.8.117, 52.217.14.68, 52.217.44.52, ...
Connecting to dataincubator-wqu.s3.amazonaws.com (dataincubator-wqu.s3.amazonaws.com)|3.5.8.117|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3398120 (3.2M) [text/csv]
Saving to: './data/providers-train.csv'

     0K .......... .......... .......... .......... ..........  1% 81.0K 40s
    50K .......... .......... .......... .......... ..........  3%  182K 29s
   100K .......... .......... .......... .......... ..........  4% 1008K 20s
   150K .......... .......... .......... .......... ..........  6%  170K 19s
   200K .......... .......... .......... .......... ..........  7% 1.11M 16s
   250K .......... .......... .......... .......... ..........  9%  200K 15s
   300K .......... .......... .......... .......... .......... 10% 1.41M 13s
   

#### We will load data into Pandas DataFrame

In [147]:
import numpy as np
import pandas as pd

In [148]:
metadata = pd.read_csv('./data/providers-metadata.csv')
metadata.head()

,Variable,Label,Description,Format
0,PROVNUM,Federal Provider Number,Federal Provider Number,6 alphanumeric characters
1,PROVNAME,Provider Name,Provider Name,text
2,ADDRESS,Provider Address,Provider Address,text
3,CITY,Provider City,Provider City,text
4,STATE,Provider State,Provider State,2-character postal abbreviation


In [149]:
train_data = pd.read_csv('./data/providers-train.csv', encoding = 'latin1')
train_data.head()
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13892 entries, 0 to 13891
Data columns (total 32 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   PROVNUM                13892 non-null  object 
 1   PROVNAME               13892 non-null  object 
 2   ADDRESS                13892 non-null  object 
 3   CITY                   13892 non-null  object 
 4   STATE                  13892 non-null  object 
 5   ZIP                    13892 non-null  int64  
 6   PHONE                  13892 non-null  int64  
 7   COUNTY_SSA             13892 non-null  int64  
 8   COUNTY_NAME            13892 non-null  object 
 9   BEDCERT                13892 non-null  int64  
 10  RESTOT                 13483 non-null  float64
 11  INHOSP                 13892 non-null  bool   
 12  CCRC_FACIL             13892 non-null  bool   
 13  SFF                    13892 non-null  bool   
 14  CHOW_LAST_12MOS        13892 non-null  bool   
 15  SP

In [150]:
fine_count = train_data.pop('FINE_CNT') 
fine_total = train_data.pop('FINE_TOT')
cycle_2_score = train_data.pop('CYCLE_2_TOTAL_SCORE')

## Question 1: state_model

A federal agency, Centers for Medicare and Medicaid Services (CMS), imposes regulations on nursing homes. However, nursing homes are inspected by state agencies for compliance with regulations, and fines for violations can vary widely between states.

Let's develop a very simple initial model to predict the amount of fines a nursing home might expect to pay based on its location. Fill in the class definition of the custom estimator, GroupMeanEstimator, below.

In [151]:
from sklearn.base import BaseEstimator, RegressorMixin, TransformerMixin

class GroupMeanEstimator(BaseEstimator, RegressorMixin):
    def __init__(self, group):
        self.group = group
        self.group_averages = {}
        self.avg = 0

    def fit(self, X, y):
        # Use self.group_averages to store the average penalty by group
        self.group_averages = y.groupby(X[self.group]).mean().to_dict()
        self.avg = y.mean()
        return self

    def predict(self, X):
        if not isinstance(X, pd.DataFrame):
            X = pd.DataFrame(X)
        # Return a list of predicted penalties based on group of samples in X
        return [self.group_averages.get(item,self.avg) for item in X[self.group]]

#### After filling in class definition, we can create an instance of the estimator and fit it to the data.

In [152]:
state = GroupMeanEstimator(group='STATE')

In [153]:
state.fit(train_data, fine_total)

GroupMeanEstimator(group='STATE')

#### Next we should test that our predict method works.

In [154]:
state.predict(train_data.sample(5))

[9216.964686998395,
 8054.977611940299,
 5626.954545454545,
 16722.25925925926,
 30445.040506329115]

#### However, what if we have data from a nursing home in a state (or territory) of the US which is not in the training data?

In [155]:
state.predict(pd.DataFrame([{'STATE': 'AS'}]))

[14969.857687877915]

## Question 2: simple_features_model

Nursing homes vary greatly in their business characteristics. Some are owned by the government or non-profits while others are run for profit. Some house a few dozen residents while others house hundreds. Some are located within hospitals and may work with more vulnerable populations. We will try to predict which facilities are fined based on their business characteristics.

We'll begin with columns in our DataFrame containing numeric and boolean features. Some of the rows contain null values; estimators cannot handle null values so these must be imputed or dropped. We will create a Pipeline containing transformers that process these features, followed by an estimator

In [156]:
simple_cols = ['BEDCERT', 'RESTOT', 'INHOSP', 'CCRC_FACIL', 'SFF', 'CHOW_LAST_12MOS', 'SPRINKLER_STATUS', 'EXP_TOTAL', 'ADJ_TOTAL']

In [157]:
class ColumnSelectTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, column):
        self.column = column
        
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        if not isinstance(X, pd.DataFrame):
            X = pd.DataFrame(X)
        return X[self.column]

In [158]:
from sklearn.impute import SimpleImputer
imp = SimpleImputer(strategy = 'mean')
imp.fit_transform(train_data[simple_cols])

array([[ 85.        ,  74.2       ,   1.        , ...,   1.        ,
          3.21218742,   3.85912129],
       [ 50.        ,  86.76046874,   1.        , ...,   1.        ,
          3.21218742,   3.85912129],
       [ 92.        ,  79.8       ,   0.        , ...,   1.        ,
          3.08015   ,   3.83026   ],
       ...,
       [ 61.        ,  86.76046874,   0.        , ...,   1.        ,
          3.21218742,   3.85912129],
       [126.        , 105.2       ,   0.        , ...,   1.        ,
          3.46718   ,   2.60224   ],
       [120.        ,  56.        ,   0.        , ...,   1.        ,
          3.49135   ,   3.14088   ]])

In [159]:
from sklearn.pipeline import Pipeline
simple_features = Pipeline([
    ('cst', ColumnSelectTransformer(simple_cols)),
    ('imp', imp)
])

In [161]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
import numpy as np
grid_param = {"C": np.logspace(-2, 2, 10)}
grid_model = RandomizedSearchCV(LogisticRegression(solver='lbfgs'), grid_param, cv=5, verbose=2)

simple_features_model = Pipeline(steps=[('simple', simple_features), 
                                        ('model', grid_model)])

In [164]:
simple_features_model.fit(train_data, fine_count > 0)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


C:\Users\DELL-INSPIRON-3576\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


[CV] END .............................................C=0.01; total time=   0.4s
[CV] END .............................................C=0.01; total time=   0.0s
[CV] END .............................................C=0.01; total time=   0.0s


C:\Users\DELL-INSPIRON-3576\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


[CV] END .............................................C=0.01; total time=   0.0s
[CV] END .............................................C=0.01; total time=   0.0s
[CV] END .............................C=0.027825594022071243; total time=   0.0s


...

[CV] END ................................C=35.93813663804626; total time=   0.0s
[CV] END ................................C=35.93813663804626; total time=   0.0s
[CV] END ............................................C=100.0; total time=   0.0s
[CV] END ............................................C=100.0; total time=   0.0s
[CV] END ............................................C=100.0; total time=   0.0s


C:\Users\DELL-INSPIRON-3576\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\DELL-INSPIRON-3576\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#log

[CV] END ............................................C=100.0; total time=   0.0s
[CV] END ............................................C=100.0; total time=   0.0s


C:\Users\DELL-INSPIRON-3576\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Pipeline(steps=[('simple',
                 Pipeline(steps=[('cst',
                                  ColumnSelectTransformer(column=['BEDCERT',
                                                                  'RESTOT',
                                                                  'INHOSP',
                                                                  'CCRC_FACIL',
                                                                  'SFF',
                                                                  'CHOW_LAST_12MOS',
                                                                  'SPRINKLER_STATUS',
                                                                  'EXP_TOTAL',
                                                                  'ADJ_TOTAL'])),
                                 ('imp', SimpleImputer())])),
                ('model',
                 RandomizedSearchCV(cv=5, estimator=LogisticRegression(),
                                    param_distributions=

In [166]:
simple_features_model.score(train_data, fine_count>0)

0.6933486898934639

## Question 3: categorical_features

The 'OWNERSHIP' and 'CERTIFICATION' columns contain categorical data. We will have to encode the categorical data into numerical features before we pass them to an estimator. Construct one or more pipelines for this purpose. Transformers such as LabelEncoder and OneHotEncoder may be useful, but you may also want to define your own transformers.

If you used more than one Pipeline, combine them with a FeatureUnion

In [205]:
imp = SimpleImputer(strategy='most_frequent')
owner_onehot = Pipeline([
    ('cst', ColumnSelectTransformer(['OWNERSHIP'])),
    ('ohe', OneHotEncoder(categories='auto', sparse = False))
])
cert_onehot = Pipeline([
    ('cst', ColumnSelectTransformer(['CERTIFICATION'])),
    ('ohe', OneHotEncoder(categories='auto', sparse = False))
])

categorical_features = FeatureUnion(
    [('owner_onehot',  owner_onehot),
    ('cert_onehot', cert_onehot)]
)

In [206]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV

n_estimators = [100, 250, 50]
max_depth = [2, 4, 6]
grid_param = {'max_depth': np.logspace(-2, 2, 10)}

grid_search_model = RandomizedSearchCV(RandomForestClassifier(n_estimators=200), grid_param, cv=5, n_jobs=2, n_iter=20, verbose=2)
categorical_features_model = Pipeline([
    ('categorical', categorical_features),
    # add your estimator here
    ('classifier', grid_search_model)
])

In [207]:
categorical_features_model.fit(train_data, fine_count>0)

C:\Users\DELL-INSPIRON-3576\anaconda3\lib\site-packages\sklearn\model_selection\_search.py:285: UserWarning: The total space of parameters 10 is smaller than n_iter=20. Running 10 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Fitting 5 folds for each of 10 candidates, totalling 50 fits


Pipeline(steps=[('categorical',
                 FeatureUnion(transformer_list=[('owner_onehot',
                                                 Pipeline(steps=[('cst',
                                                                  ColumnSelectTransformer(column=['OWNERSHIP'])),
                                                                 ('ohe',
                                                                  OneHotEncoder(sparse=False))])),
                                                ('cert_onehot',
                                                 Pipeline(steps=[('cst',
                                                                  ColumnSelectTransformer(column=['CERTIFICATION'])),
                                                                 ('ohe',
                                                                  OneHotEncoder(sparse=False))]))])),
                ('classifier',
                 RandomizedSearchCV(cv=5,
                                    estim

In [208]:
categorical_features_model.score(train_data, fine_count>0)

0.6922689317592859

# Question 4: business_model

Finally, we'll combine simple_features and categorical_features in a FeatureUnion, followed by an estimator in a Pipeline. You may want to optimize the hyperparameters of your estimator using cross-validation or try engineering new features (e.g. see PolynomialFeatures). When you've assembled and trained your model, pass the predict_proba method to the grader.

In [219]:
business_features = FeatureUnion([
    ('simple', simple_features),
    ('categorical', categorical_features)
])

In [220]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
import numpy as np
grid_param = {"C": np.logspace(-2, 2, 10)}
grid_model = RandomizedSearchCV(LogisticRegression(), grid_param, cv=5, verbose=2)

business_model = Pipeline([
    ('features', business_features),
    # add your estimator here
    ('poly', PolynomialFeatures(2)),
    ('model', grid_model)
])

In [221]:
business_model.fit(train_data, fine_count > 0)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


C:\Users\DELL-INSPIRON-3576\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


[CV] END .............................................C=0.01; total time=   0.5s


C:\Users\DELL-INSPIRON-3576\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


[CV] END .............................................C=0.01; total time=   0.5s


...

[CV] END ............................................C=100.0; total time=   0.5s


C:\Users\DELL-INSPIRON-3576\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


[CV] END ............................................C=100.0; total time=   0.5s


C:\Users\DELL-INSPIRON-3576\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Pipeline(steps=[('features',
                 FeatureUnion(transformer_list=[('simple',
                                                 Pipeline(steps=[('cst',
                                                                  ColumnSelectTransformer(column=['BEDCERT',
                                                                                                  'RESTOT',
                                                                                                  'INHOSP',
                                                                                                  'CCRC_FACIL',
                                                                                                  'SFF',
                                                                                                  'CHOW_LAST_12MOS',
                                                                                                  'SPRINKLER_STATUS',
                                                              

In [222]:
business_model.score(train_data, fine_count > 0)

0.6886697379786928

## Question 5: survey_results

Surveys reveal safety and health deficiencies at nursing homes that may indicate risk for incidents (and penalties). CMS routinely makes surveys of nursing homes. Build a model that combines the business_features of each facility with its cycle 1 survey results, as well as the time between the cycle 1 and cycle 2 survey to predict the cycle 2 total score.

First, let's create a transformer to calculate the difference in time between the cycle 1 and cycle 2 surveys.

In [240]:
class TimedeltaTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, t1_col, t2_col):
        self.t1_col = t1_col
        self.t2_col = t2_col
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        if not isinstance(X, pd.DataFrame):
            X = pd.DataFrame(X)
        delta = pd.to_datetime(X[self.t2_col]) - pd.to_datetime(X[self.t1_col]) 
        num_days = delta.apply(lambda X:X.days).values.reshape(-1,1)
        return num_days

In [241]:
cycle_1_date = 'CYCLE_1_SURVEY_DATE'
cycle_2_date = 'CYCLE_2_SURVEY_DATE'
time_feature = TimedeltaTransformer(cycle_1_date,cycle_2_date)

In the cell below we'll collect the cycle 1 survey features.

In [242]:
cycle_1_cols = ['CYCLE_1_DEFS', 'CYCLE_1_NFROMDEFS', 'CYCLE_1_NFROMCOMP',
                'CYCLE_1_DEFS_SCORE', 'CYCLE_1_NUMREVIS',
                'CYCLE_1_REVISIT_SCORE', 'CYCLE_1_TOTAL_SCORE']
cycle_1_features = ColumnSelectTransformer(cycle_1_cols)

In [243]:
from sklearn.pipeline import FeatureUnion
from sklearn.preprocessing import OneHotEncoder

In [244]:
feature = FeatureUnion([
    ('business', business_features),
    ('survey', cycle_1_features),
    ('time', time_feature)
])

In [245]:
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.linear_model import Lasso

grid_search = GridSearchCV(Lasso(max_iter = 100), param_grid = {'alpha': np.arange(0, 3.5, 0.5)}, cv=5, n_jobs=4, verbose=1)
survey_model = Pipeline([
    ('features', feature),
    ('poly', PolynomialFeatures(2)),
    ('decomp', TruncatedSVD(40)),
    ('grid_search', grid_search)
])

In [246]:
survey_model.fit(train_data, fine_count)

Fitting 5 folds for each of 7 candidates, totalling 35 fits


Pipeline(steps=[('features',
                 FeatureUnion(transformer_list=[('business',
                                                 FeatureUnion(transformer_list=[('simple',
                                                                                 Pipeline(steps=[('cst',
                                                                                                  ColumnSelectTransformer(column=['BEDCERT',
                                                                                                                                  'RESTOT',
                                                                                                                                  'INHOSP',
                                                                                                                                  'CCRC_FACIL',
                                                                                                                                  'SFF',
            

In [247]:
survey_model.score(train_data, fine_count)

0.2259220084643221